In [1]:
#url 파싱 
 #scheme,hostname만 알면된다
 
from urllib.parse import urlparse

u = urlparse('https://www.google.com') #url

print(u.scheme)
print(u.hostname)

https
www.google.com


In [2]:
#TCP만 연결 -SSL 없이 보안 없이
import socket
#세로운 메서드 socket에서 
#create_connection((hostname,port))
with socket.create_connection((u.hostname,443)) as sock:
    print("TCP connected")
    print(sock)

TCP connected
<socket.socket fd=1400, family=AddressFamily.AF_INET6, type=SocketKind.SOCK_STREAM, proto=0, laddr=('2406:5900:101a:4c56:9450:2696:538f:2693', 54178, 0, 0), raddr=('2404:6800:4005:822::2004', 443, 0, 0)>


In [3]:
#3) TLS 컨텍스트 만들기 (검증옵션)

import ssl

ctx = ssl.create_default_context()
#20살이 민증들고 술사려고 가 민증검사를 해야해 일단 이민증이 올바르게 국가에서 발급이 되었는지 눈으로봐야함ssl.CERT_REQUIRED  그다음에 이사람 민증 얼굴을 대조해서 맞는지도 봐야함 check_hostname 

ctx.check_hostname = True  # 접속 호스트 명과 인증서 이름 일치 검사
ctx.verify_mode = ssl.CERT_REQUIRED # 인증서가 신뢰 가능한 ca에서 발급 되었는지 검사


#목적 보안 통신 규칙 준비

In [9]:
#TLS로 감싸기

import socket,ssl

hostname = "www.google.com"
ctx = ssl.create_default_context() #검증도구

#ctx.check_hostname = True 
#ctx.verify_mode = ssl.CERT_REQUIRED

with socket.create_connection((hostname,443))as sock:
    with ctx.wrap_socket(sock,server_hostname=hostname) as ssock: 
        cert = ssock.getpeercert() 
        print(cert)

{'subject': ((('commonName', 'www.google.com'),),), 'issuer': ((('countryName', 'US'),), (('organizationName', 'Google Trust Services'),), (('commonName', 'WE2'),)), 'version': 3, 'serialNumber': '0A8D497AAC4C4D9410A6911C22EE555E', 'notBefore': 'Jul  7 08:36:00 2025 GMT', 'notAfter': 'Sep 29 08:35:59 2025 GMT', 'subjectAltName': (('DNS', 'www.google.com'),), 'OCSP': ('http://o.pki.goog/we2',), 'caIssuers': ('http://i.pki.goog/we2.crt',), 'crlDistributionPoints': ('http://c.pki.goog/we2/dTM3-0hpWfE.crl',)}


In [10]:
#cn 공통이름 뽑기 subject,issuer

def pick_cn(name_list):
    
    for tup in name_list:
        for k,v in tup:
            if k.lower() =="commonname":
                return v
    return None

subject_cn =pick_cn(cert.get("subject"))
issuer_cn = pick_cn(cert.get("issuer"))

print(subject_cn)
print(issuer_cn)
            

www.google.com
WE2


In [11]:
sans = [v for k,v in cert.get("subjectAltName",())if k=="DNS"]
print(sans)

['www.google.com']


In [12]:
cert.get( "subjectAltName"
)

(('DNS', 'www.google.com'),)

In [17]:
#만료일파싱 -> 시간비교 (형식주의)

from datetime import timezone,datetime

not_after = cert.get("notAfter")

dt = datetime.strptime(not_after,"%b %d %H:%M:%S %Y %Z").replace(tzinfo=timezone.utc)

expired= dt < datetime.now(timezone.utc)

print(dt.isoformat()) 
print(expired)

2025-09-29T08:35:59+00:00
False


In [19]:
issuer_cn = pick_cn(cert.get("issuer",()))  
print(issuer_cn)

WE2


In [20]:
result ={
    "valid": not expired,
    "subject_cn": subject_cn,
    "issuer_cn": issuer_cn,
    "not_after": dt.isoformat(),  
} 
print(result)

{'valid': True, 'subject_cn': 'www.google.com', 'issuer_cn': 'WE2', 'not_after': '2025-09-29T08:35:59+00:00'}
